## Below is the same code as Example_1_toymodel.py
## It is broken out into a Jupyter Notebook 

In [1]:
import mfapy

ModuleNotFoundError: No module named 'mfapy'

In [2]:
# Model construction
reactions, reversible, metabolites, fragments\
 = mfapy.mfapyio.load_metabolic_model("example_1_toymodel_model.txt")
model = mfapy.metabolicmodel.MetabolicModel(reactions, reversible,\
 metabolites, fragments)

In [3]:
# Addition of constraints
state = model.load_states("Example_1_toymodel_status.csv", format = 'csv')
model.set_constraints_from_state_dict(state)
model.update()

True

In [4]:
# Generation of CarbonSource instance
cs1 = model.generate_carbon_source_template()
cs1.set_each_isotopomer('AcCoA', {'#10':0.5})
cs2 = model.generate_carbon_source_template()
cs2.set_each_isotopomer('AcCoA', {'#11':0.5})

True

In [5]:
# Load MDV data
mdv1 = model.load_mdv_data("Example_1_MDV1.txt")
mdv2 = model.load_mdv_data("Example_1_MDV2.txt")

In [6]:
# Flux estimation Step 1: Setting experimments
model.set_experiment('ex1', mdv1, cs1)
model.set_experiment('ex2', mdv2, cs2)

True

In [7]:
# Flux estimation step 2: Generation of intical flux vectors
endstate, flux = model.generate_initial_states(50, 4, method ="parallel")

In [8]:
# Flux estimation step 3: Fitting model
for method in ["GN_CRS2_LM", "LN_PRAXIS", "SLSQP"]:
    endstate, RSS, flux = model.fitting_flux(method = method, flux = flux)
model.show_results([("final", flux[0])]) # Show result

Id             Reaction                 External i   final
RSS                                                   8.23
Thres                                                15.51
p_value                                               0.41
Id             Reaction                 External i   final Type     Value Stdev    lb    ub Atom_mapping             
v1             AcCoA+OAC-->Cit          (kegg:R003   100.0 fixed    100.0   1.0   0.0 300.0 AB+CDEF-->FEDBAC         
v2             Cit-->AKG+CO2ex          (kegg:R007   100.0 free     100.0   1.0   0.0 300.0 ABCDEF-->ABCDE+F         
v3             AKG-->Glu                (kegg:R002    49.7 free      50.0   1.0   0.0 300.0 ABCDE-->ABCDE            
v4             AKG-->Suc+CO2ex          (kegg:R011    50.3 free      50.0   1.0   0.0 300.0 ABCDE-->BCDE+A           
v5             Suc-->Fum                (kegg:R021    50.3 free      50.0   1.0   0.0 300.0 ABCD-->ABCD              
v6             Fum-->OAC                (kegg:R010   247

In [9]:
# Estimation of 95% CI
ci_edge = model.generate_ci_template(targets = [('reaction', "v3")])
ci = model.search_ci(ci_edge, flux[0], method = 'grid')
lb = ci['data'][('reaction', "v3")]['lower_boundary']
ub = ci['data'][('reaction', "v3")]['upper_boundary']
print("v3", "Lower bondary:",lb, "Upper boundary:", ub)


v3 Lower bondary: 48.12866588797964 Upper boundary: 51.368283728679565


In [11]:
# Estimation of 95% CI
ci_edge = model.generate_ci_template(targets = [('reaction', "v6")])
ci = model.search_ci(ci_edge, flux[0], method = 'grid')
lb = ci['data'][('reaction', "v6")]['lower_boundary']
ub = ci['data'][('reaction', "v6")]['upper_boundary']
print("v6", "Lower bondary:",lb, "Upper boundary:", ub)


LinAlgError: Singular matrix